In [19]:
import json
import os
import pandas as pd

In [20]:
dir = "../../openneuro_extracts"

In [21]:
data ={}
for root, dirs, files in os.walk(dir):
    for file in files:
        with open(f"{root}/{file}") as f:
            data.update(json.load(f))


In [22]:
df = pd.DataFrame.from_dict(data, orient = "index")


In [23]:
useful = df[(df["SeriesDescription"] != "NA") | (df["ProtocolName"] != "NA")]

useful.reset_index(inplace=True)


In [24]:
useful["index"] = useful["index"].apply(lambda x: x.rsplit("_", 1)[-1])
useful["index"] = useful["index"].apply(lambda x: x.split(".")[0])

/tmp/ipykernel_599131/2899444110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful["index"] = useful["index"].apply(lambda x: x.rsplit("_", 1)[-1])
/tmp/ipykernel_599131/2899444110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful["index"] = useful["index"].apply(lambda x: x.split(".")[0])


In [25]:
useful.drop_duplicates(subset=["index", "SeriesDescription", "ProtocolName"], inplace=True)

/tmp/ipykernel_599131/1737227950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful.drop_duplicates(subset=["index", "SeriesDescription", "ProtocolName"], inplace=True)


In [26]:
suffixes = ['T1w','T2w','dwi','bold']

In [27]:
filtered = useful[useful['index'].isin(suffixes)]

In [28]:
filtered

,index,SeriesDescription,ProtocolName,RepetitionTime,EchoTime,InversionTime,PulseSequenceType,FlipAngle,Manufacturer,ManufacturersModelName
0,bold,BOLD IPAT2 BART,NA,2,0.03,0,NA,90,SIEMENS,TrioTim
1,bold,BOLD EPI - Stopmanual2,NA,2,0.03,0.0,NA,90.0,SIEMENS,Allegra
2,bold,BOLD EPI - Stopmanual1,NA,2,0.03,0.0,NA,90.0,SIEMENS,Allegra
3,bold,BOLD EPI - Stopword1,NA,2,0.03,0.0,NA,90.0,SIEMENS,Allegra
4,bold,BOLD EPI - Stopword2,NA,2,0.03,0.0,NA,90.0,SIEMENS,Allegra
...,...,...,...,...,...,...,...,...,...,...
159402,bold,VOE1_SMS2_3mm_iso,VOE1_SMS2_3mm_iso,2,0.03,NA,NA,90,Siemens,Prisma_fit
159403,bold,VOE4_SMS2_3mm_iso,VOE4_SMS2_3mm_iso,2,0.03,NA,NA,90,Siemens,Prisma_fit
159404,bold,DOTSLoc1_SMS2_3mm_iso,DOTSLoc1_SMS2_3mm_iso,2,0.03,NA,NA,90,Siemens,Prisma_fit
159409,bold,DOTS1_SMS2_3mm_iso,DOTS1_SMS2_3mm_iso,2,0.03,NA,NA,90,Siemens,Prisma


In [31]:
filtered.to_json("examples_simple.json",orient = 'records', indent =4)

In [30]:
filtered['index'].value_counts()

index
bold    2778
T1w      526
dwi      197
T2w      116
Name: count, dtype: int64

In [69]:
filtered['ProtocolName'].value_counts()

ProtocolName
NA                           167
ADNI3 FMRIF Enhanced Pro      38
FMRIF - Clinical 32Ch Br      26
SLP - 1-second Memory (2      13
[XT-ID:01-M-0192] MAP-3P      12
                            ... 
EPI_run11_204dyn               1
sT1W_1mm_3D_32ch               1
EPI_LOCALIZER_56dyn_SENSE      1
EPI_run6_195dyn_SENSE          1
EPI_run9_204dyn                1
Name: count, Length: 3890, dtype: int64

In [70]:
dropped = filtered.dropna()

In [73]:
import numpy as np

In [74]:
dropped.replace("NA",np.nan, inplace =True)


/tmp/ipykernel_444717/2496649854.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dropped.replace("NA",np.nan, inplace =True)


In [14]:
filtered['index'].value_counts()

index
bold          2778
T1w            526
epi            482
dwi            197
T2w            116
FLAIR          105
phasediff       49
magnitude1      49
magnitude2      43
magnitude       12
asl             12
phase2           5
phase1           5
Name: count, dtype: int64

In [76]:
dropped

,index,SeriesDescription,ProtocolName,RepetitionTime,EchoTime,InversionTime,PulseSequenceType,FlipAngle,Manufacturer,ManufacturersModelName


In [2]:
import json
import pandas as pd

outputs = "../model_outputs_gpt4o.json"
with open(outputs) as f:
    dic = json.load(f)


In [3]:
df4 = pd.DataFrame(dic)

In [4]:
df4['correct'] = df4['suffix'].str.lower() == df4['actual'].str.lower()

In [5]:
df4['correct'].value_counts()

correct
True     1400
False      46
Name: count, dtype: int64

In [8]:
incorrect = df4[df4['correct']==False]

In [21]:
incorrect['actual'].value_counts()

actual
bold    33
T1w      9
dwi      2
T2w      2
Name: count, dtype: int64

In [12]:
conts =incorrect.groupby(['series_description', 'protocol_name']).size().reset_index(name='counts')

In [20]:
incorrect[incorrect['suffix'] == 'T1w']

,suffix,series_description,protocol_name,task_name,repetition_time,echo_time,inversion_time,pulse_sequence_type,flip_angle,manufacturer,...,additional_kwargs,response_metadata,type,name,id,example,tool_calls,invalid_tool_calls,usage_metadata,correct
22,T1w,Slab00 Echo00 Mag Combined Coils,pgrs3d ey v15 ipat1 centric no FS long RF2 run4,NA,NaN,0.029210,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
31,T1w,fft40 1.0iso FA17,fft40 1.0iso FA17,NA,NaN,0.002670,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
129,T1w,Slab00 Echo00 Mag Combined Coils,pgrs3d ey v15 ipat1 centric run1,NA,NaN,0.027930,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
144,T1w,NA,WIPANT ASENSE,NA,NaN,0.025000,NaN,NA,NaN,Philips,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
195,T1w,Slab00 Echo00 Mag Combined Coils,pgrs3d ey v15 ipat1 centric no FS long RF2,NA,NaN,0.029210,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
239,T1w,grs3d v16 vfa no PF Hi Gain 85v kfp4,grs3d v16 vfa no PF Hi Gain 85v kfp4,NA,NaN,0.033430,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
254,T1w,Slab00 Echo00 Mag Combined Coils,pgrs3d ey v15 ipat1 centric run3,NA,NaN,0.027930,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
432,T1w,NA,4 1 2 anatomical 3d highres MB,NA,NaN,0.040076,NaN,NA,NaN,Bruker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
683,T1w,Slab00 Echo00 Mag Combined Coils,pgrs3d ey v15 ipat1 centric no FS run7,NA,NaN,0.027930,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
794,T1w,grs3d v16 vfa no PF Hi Gain 85v kfp3,grs3d v16 vfa no PF Hi Gain 85v kfp3,NA,NaN,0.033430,NaN,NA,NaN,Siemens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
